In [1]:
%load_ext autoreload
%autoreload 2

import base64
import imageio
import IPython
import numpy as np
import PIL.Image
import pyvirtualdisplay
import gym
import gym_battleship
import tensorflow as tf

from collections import OrderedDict
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.utils import common
from agents.train_agent import train_agent
from models.residual_cnn import create_res_cnn_model
from helpers.random_hyper_picker import pick_random_hyper_parameters
from models.fc_net import create_fc_model
from models.cnn import create_cnn_model
from tensorflow.python.keras.optimizer_v2.learning_rate_schedule import PolynomialDecay
import os
from datetime import datetime
import json
from models.cnn import validate_model_parameters

In [2]:
# m_22 = {'id': 22, 'best_reward': -842.8, 'last_5_reward_avg': -871.6, 'reward_variance': 1706.9448, 'best_steps': 100.0, 'last_5_avg_steps': 100.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 200000), ('mask_invalid_actions', True), ('initial_collect_steps', 10000), ('starting_epsilon', 0.1), ('ending_epsilon', 0.001), ('learning_rate', 1e-05), ('learning_rate_decay', 0.90), ('batch_size', 512), ('target_update_tau', 0.01), ('model', {'activation': 'relu', 'batch_norm': True, 'dropout': 0.4, 'l2_regularisation': False, 'feed_input_to_last_layer': True, 'condense_sunk_matrix': True, 'modules': [{'type': 'conv', 'filter_size': (3, 3), 'filter_count': 128}, {'type': 'conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3)}], 'fc_layers': [64, 32]})])}
#
# # m_23 = {'id': 23, 'best_reward': -830.2, 'last_5_reward_avg': -887.5, 'reward_variance': 1464.74, 'best_steps': 100.0, 'last_5_avg_steps': 100.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 200000), ('constrain_action_space', False), ('initial_collect_steps', 50000), ('starting_epsilon', 0.2), ('ending_epsilon', 0.001), ('learning_rate', 1e-05), ('learning_rate_decay', 0.99), ('batch_size', 1024), ('target_update_tau', 0.01), ('model', {'activation': 'relu', 'batch_norm': True, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': True, 'condense_sunk_matrix': True, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (2, 2), 'filter_count': 32, 'pool_func': 'avg', 'pool_size': (3, 3), 'pad': 2}, {'type': 'pad-conv-pool', 'filter_size': (2, 2), 'filter_count': 32, 'pool_func': 'avg', 'pool_size': (2, 2), 'pad': 1}], 'fc_layers': []})])}
#
# m_22_var = {'id': 2201, 'best_reward': -842.8, 'last_5_reward_avg': -871.6, 'reward_variance': 1706.9448, 'best_steps': 100.0, 'last_5_avg_steps': 100.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('constrain_action_space', False), ('initial_collect_steps', 100), ('starting_epsilon', 0.1), ('ending_epsilon', 0.001), ('learning_rate', 1e-05), ('learning_rate_decay', 0.99), ('batch_size', 16), ('target_update_tau', 0.01), ('model', {'activation': 'relu', 'batch_norm': True, 'dropout': 0.4, 'l2_regularisation': False, 'feed_input_to_last_layer': True, 'condense_sunk_matrix': True, 'modules': [{'type': 'conv', 'filter_size': (3, 3), 'filter_count': 128}, {'type': 'conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3)}], 'fc_layers': [64, 32]})])}
#

# m_16 = {'id': 16, 'best_reward': 116.1, 'last_5_reward_avg': 106.7, 'reward_variance': 13.838369, 'best_steps': 96.9, 'last_5_avg_steps': 90.3, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 200000), ('mask_invalid_actions', True), ('initial_collect_steps', 50000), ('starting_epsilon', 0.2), ('ending_epsilon', 0.05), ('learning_rate', 0.0001), ('learning_rate_decay', 0.95), ('batch_size', 256), ('target_update_tau', 0.0001), ('model', {'activation': 'relu', 'batch_norm': True, 'dropout': 0.4, 'l2_regularisation': False, 'feed_input_to_last_layer': True, 'condense_sunk_matrix': True, 'modules': [{'type': 'conv-pool', 'filter_size': (4, 4), 'filter_count': 32, 'pool_func': 'max', 'pool_size': (2, 2)}, {'type': 'pad-conv-pool', 'filter_size': (1, 1), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 3}, {'type': 'conv', 'filter_size': (1, 1), 'filter_count': 128}], 'fc_layers': [16, 16]})])}
#
# m_new = {'id': 99, 'hyper_parameters': OrderedDict([('num_iterations', 50000),('gamma',0.9), ('replay_buffer_max_length', 2000), ('mask_invalid_actions', True), ('initial_collect_steps', 1000), ('starting_epsilon', 1.0), ('ending_epsilon', 0.01), ('learning_rate', 0.0001), ('learning_rate_decay', 0.99), ('batch_size', 16), ('target_update_tau', 0.1), ('model', {'activation': 'relu', 'batch_norm': True, 'dropout': 0, 'l2_regularisation': False, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'modules': [{'type': 'pad-conv', 'filter_size': (7, 7), 'filter_count': 32, 'pad': 3},{'type': 'pad-conv-pool', 'filter_size': (7, 7), 'filter_count': 32, 'pool_func': 'max', 'pool_size': (2, 2), 'pad': 3}], 'fc_layers': []})])}


# m_13 = {'id': 13, 'best_reward': 117.4, 'last_5_reward_avg': 116.8, 'reward_variance': 24.341602, 'best_steps': 98.2, 'last_5_avg_steps': 80.2, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('initial_collect_steps', 500), ('mask_invalid_actions', True), ('gamma', 0.99), ('starting_epsilon', 0.1), ('ending_epsilon', 0.001), ('learning_rate', 1e-05), ('learning_rate_decay', 0.98), ('batch_size', 64), ('target_update_tau', 0.0001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': False, 'feed_input_to_last_layer': True, 'condense_sunk_matrix': False, 'modules': [{'type': 'conv-pool', 'filter_size': (1, 1), 'filter_count': 128, 'pool_func': 'max', 'pool_size': (2, 2)}, {'type': 'pad-conv-pool', 'filter_size': (1, 1), 'filter_count': 64, 'pool_func': 'avg', 'pool_size': (2, 2), 'pad': 0}], 'fc_layers': []})])}

# m_14 = {'id': 14, 'best_reward': 117.4, 'last_5_reward_avg': 107.0, 'reward_variance': 14.316982, 'best_steps': 96.6, 'last_5_avg_steps': 90.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('initial_collect_steps', 500), ('mask_invalid_actions', True), ('gamma', 0.95), ('starting_epsilon', 0.2), ('ending_epsilon', 0.001), ('learning_rate', 0.001), ('learning_rate_decay', 0.98), ('batch_size', 16), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0.2, 'l2_regularisation': False, 'feed_input_to_last_layer': True, 'condense_sunk_matrix': False, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (2, 2), 'filter_count': 128, 'pool_func': 'avg', 'pool_size': (2, 2), 'pad': 2}, {'type': 'conv', 'filter_size': (2, 2), 'filter_count': 128}], 'fc_layers': [32, 16]})])}

m_7 = {'id': 7, 'best_reward': 115.0, 'last_5_reward_avg': 103.0, 'reward_variance': 17.83225, 'best_steps': 98.0, 'last_5_avg_steps': 94.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('initial_collect_steps', 500), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 1), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': False,'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': []})])}

# Toggle additional layer
m_7_var1 = {'id': 7, 'best_reward': 115.0, 'last_5_reward_avg': 103.0, 'reward_variance': 17.83225, 'best_steps': 98.0, 'last_5_avg_steps': 94.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('initial_collect_steps', 500), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 1), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': True, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': [128]})])}

# Increase buffer size
m_7_var2 = {'id': 7, 'best_reward': 115.0, 'last_5_reward_avg': 103.0, 'reward_variance': 17.83225, 'best_steps': 98.0, 'last_5_avg_steps': 94.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 200_000), ('initial_collect_steps', 50_000), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 1),('decay_every',50_000), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': False, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': [128]})])}

# Additional 2 layers
m_7_var3 = {'id': 7, 'best_reward': 115.0, 'last_5_reward_avg': 103.0, 'reward_variance': 17.83225, 'best_steps': 98.0, 'last_5_avg_steps': 94.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('initial_collect_steps', 500), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 1),('decay_every',50_000), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': False, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': [128,128]})])}

# Additional 2 layers with no regularisation loss
m_7_var4 = {'id': 7, 'best_reward': 115.0, 'last_5_reward_avg': 103.0, 'reward_variance': 17.83225, 'best_steps': 98.0, 'last_5_avg_steps': 94.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('initial_collect_steps', 500), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 1),('decay_every',50_000), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': False, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': True, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': [128,128]})])}

# TODO try with slight LR decay
m_7_var5 = {'id': 7, 'best_reward': 115.0, 'last_5_reward_avg': 103.0, 'reward_variance': 17.83225, 'best_steps': 98.0, 'last_5_avg_steps': 94.0, 'hyper_parameters': OrderedDict([('num_iterations', 50000), ('replay_buffer_max_length', 2000), ('initial_collect_steps', 500), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 0.99), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': True, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': [128]})])}


promising_configurations = [m_7_var2]#, m_7_var3, m_7_var4, m_7_var5]


# Other Prams
P = {
    'collect_steps_per_iteration': 1,
    'log_interval': 200,
    'num_eval_episodes': 5,
    'eval_interval': 100,
    'keep_top_k': 10,
    'run_alias': '10x10-promising-cnns',
    'save_hyper_parameters': True
}

# Environment Params
board_size = (10, 10)
ship_sizes = {5: 1, 4: 1, 3: 2, 2: 1}

reward_dictionary = {
    'win': 100,  # for sinking all ships
    'missed': -1,  # for missing a shot
    'hit': 5,  # for hitting a ship
    'repeat_missed': -10,  # for shooting at an already missed cell
    'repeat_hit': -10  # for shooting at an already hit cell
}

In [3]:
top_k_models = []
already_chosen_hypers = set()
debugging = True
iterations = 1000

date_string = datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
output_folder = f"data/1_tuning/{date_string}-{P['run_alias']}"
log_folder = f'{output_folder}/logs'
policy_folder = f'{output_folder}/policies'

for p in [output_folder, log_folder, policy_folder]:
    if not os.path.exists(p):
        os.mkdir(p)


for t, params in enumerate(promising_configurations):
    print(f"\n--- Inspection Run {t + 1}/{len(promising_configurations)} ---\n")
    H = params['hyper_parameters'] | {'num_iterations':iterations}
    M = H['model']
    E = {
        'ship_sizes': ship_sizes,
        'board_size': board_size,
        'episode_steps': np.prod(board_size),  # Number of steps until the episode terminates
        'reward_dictionary': reward_dictionary,
        'get_invalid_action_mask': H['mask_invalid_actions']
    }
    env = suite_gym.load('Battleship-v0', gym_kwargs=E)
    print(f'Picked hyper-parameters:')
    print(H)


    # Sometimes tf will fail with NaN values, interrupting tuning.
    # And on other occasions, the generated CNN may not be compatible with the input size.
    try:
        observation_shape = env.observation_space.spaces['observation'].shape if H['mask_invalid_actions'] else env.observation_space.shape
        validate_model_parameters(model_params=M, observation_space=observation_shape)
        q_net = create_cnn_model(model_params=M, num_actions=env.action_space.n,
                                 observation_space=observation_shape)

        agent, rewards, steps = train_agent(id=t+1, q_net=q_net, hyper_parameters=H, environment_parameters=E,
                                            other_parameters=P,
                                            policy_dir=policy_folder, log_dir=log_folder)
        last_5_avg_reward = np.mean(rewards[-5])
        best_reward = np.max(rewards)
        reward_variance = np.var(rewards)
        last_5_avg_steps = np.mean(steps[-5])
        best_steps = np.min(steps)

        top_k_models.append(
            {'id': t+1,
             'best_reward': best_reward, 'last_5_reward_avg': last_5_avg_reward, 'reward_variance': reward_variance,
             'best_steps': best_steps, 'last_5_avg_steps': last_5_avg_steps,
             'hyper_parameters': H, 'net': q_net, 'agent': agent})

        if len(top_k_models) > P['keep_top_k']:
            top_k_models = sorted(top_k_models, key=lambda x: x['best_steps'])[:-1]
            steps = [x['best_steps'] for x in top_k_models]
            print(
                f"\nTop {P['keep_top_k']} model best step stats - mean: {np.mean(steps)}, min: {np.min(steps)}, max: {np.max(steps)}")

        if P['save_hyper_parameters']:
            # Save everything except for the net to a text file
            top_k_models = sorted(top_k_models, key=lambda x: x['best_steps'])
            top_params = [str(dict(filter(lambda x: x[0] not in ['net', 'agent'], d.items()))) for d in
                          top_k_models]
            file_path = f"{output_folder}/top_parameters.json"
            with open(file_path, 'w') as f:
                f.write('\n'.join(top_params))
                print(f"Saving top {len(top_k_models)} model hyper parameters to '{file_path}'")

    except Exception as e:
        if not debugging:
            print(e)
        else:
            raise e

print('\nTop Model:')
print(f"best steps: {top_k_models[0]['best_steps']}, last 5 avg: {top_k_models[0]['last_5_avg_steps']}")
print('Hyper_parameters: ')
print(top_k_models[0]['hyper_parameters'])
print('Structure')
top_k_models[0]['net'].summary()


--- Inspection Run 1/1 ---

Picked hyper-parameters:
OrderedDict([('num_iterations', 1000), ('replay_buffer_max_length', 200000), ('initial_collect_steps', 50000), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 1), ('decay_every', 50000), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': False, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': [128]})])


2022-03-15 01:47:50.150626: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 01:47:50.161713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 01:47:50.162233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 01:47:50.163868: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24979) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24979) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24979) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24979) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24979) so Table uniform_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24979) so Table uniform_table is accessed directly without gRPC.


step = 100, Average Steps = 98.0, Avg Reward = 99.0
step = 200, loss = 164.31529235839844
step = 200, Average Steps = 95.2, Avg Reward = 101.80000305175781
step = 300, Average Steps = 91.2, Avg Reward = 105.80000305175781
step = 400, loss = 5.202773094177246
step = 400, Average Steps = 95.0, Avg Reward = 102.0
step = 500, Average Steps = 87.4, Avg Reward = 109.5999984741211
step = 600, loss = 5.208201885223389
step = 600, Average Steps = 89.6, Avg Reward = 107.4000015258789
step = 700, Average Steps = 89.2, Avg Reward = 107.80000305175781
step = 800, loss = 159.32400512695312
step = 800, Average Steps = 95.6, Avg Reward = 101.4000015258789
step = 900, Average Steps = 92.8, Avg Reward = 104.19999694824219
step = 1000, loss = 9.633711814880371
step = 1000, Average Steps = 95.2, Avg Reward = 101.80000305175781


[reverb/cc/platform/default/server.cc:84] Shutting down replay server


Saving top 1 model hyper parameters to 'data/1_tuning/2022-03-15T01-47-50-10x10-promising-cnns/top_parameters.json'

Top Model:
best steps: 87.4, last 5 avg: 89.6
Hyper_parameters: 
OrderedDict([('num_iterations', 1000), ('replay_buffer_max_length', 200000), ('initial_collect_steps', 50000), ('mask_invalid_actions', True), ('gamma', 0.9), ('starting_epsilon', 1.0), ('ending_epsilon', 0.001), ('learning_rate', 0.0001), ('learning_rate_decay', 1), ('decay_every', 50000), ('batch_size', 64), ('target_update_tau', 0.001), ('model', {'activation': 'relu', 'batch_norm': False, 'dropout': 0, 'l2_regularisation': True, 'feed_input_to_last_layer': False, 'condense_sunk_matrix': False, 'feed_input_after_cnn_layers': False, 'modules': [{'type': 'pad-conv-pool', 'filter_size': (3, 3), 'filter_count': 64, 'pool_func': 'max', 'pool_size': (3, 3), 'pad': 2}, {'type': 'conv', 'filter_size': (4, 4), 'filter_count': 64}], 'fc_layers': [128]})])
Structure
Model: "sequential"
_____________________________

In [4]:
# Play example game
eval_py_env = suite_gym.load('Battleship-v0', gym_kwargs=E)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)
#agent = top_k_models[0]['agent']
saved_policy = tf.compat.v2.saved_model.load(f"data/1_tuning/2022-03-14T12-23-27-10x10-promising-cnns/policies/1")
policy_state = saved_policy.get_initial_state(batch_size=1)

In [5]:
time_step = eval_env.reset()
counter = 0
while not time_step.is_last():
    action_step = saved_policy.action(time_step)
    time_step = eval_env.step(action_step.action)
    eval_py_env.render()
    counter +=1

eval_py_env.render_board_generated()
print(f'Completed in {counter} steps')

,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
8,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
10,⬜,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,🞇,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
8,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
10,⬜,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,❌,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
8,❌,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
10,⬜,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,❌,⬜,🞇,⬜,⬜,⬜,⬜,⬜,⬜,⬜
8,❌,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
10,⬜,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,❌,⬜,🞇,⬜,⬜,⬜,⬜,⬜,⬜,⬜
8,❌,⬜,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
10,⬜,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


...

,A,B,C,D,E,F,G,H,I,J
1,⬜,⚫,⚫,⚫,⚫,⚫,⬜,⬜,⚫,⚫
2,⚫,❌,⚫,⚫,⚫,⚫,⬜,⬜,⚫,⚫
3,⚫,❌,❌,⚫,🞇,🞇,⬜,⬜,⚫,⚫
4,⚫,❌,❌,⚫,⚫,⚫,⬜,⬜,⬜,⬜
5,⚫,⚫,❌,⚫,⚫,⚫,⬜,⬜,⬜,⚫
6,⚫,⚫,❌,⚫,⚫,⚫,⚫,⚫,⬜,⚫
7,❌,⚫,❌,⚫,❌,❌,❌,⚫,⚫,⚫
8,❌,⚫,⚫,⚫,⚫,⚫,⚫,⚫,⚫,⚫
9,⚫,⬜,⚫,⚫,⬜,⚫,⚫,⚫,⬜,⬜
10,⚫,⚫,⚫,⬜,⚫,⬜,⬜,⬜,⚫,⚫


,A,B,C,D,E,F,G,H,I,J
1,⬜,⚫,⚫,⚫,⚫,⚫,⬜,⬜,⚫,⚫
2,⚫,❌,⚫,⚫,⚫,⚫,⬜,⬜,⚫,⚫
3,⚫,❌,❌,⚫,🞇,🞇,🞇,⬜,⚫,⚫
4,⚫,❌,❌,⚫,⚫,⚫,⬜,⬜,⬜,⬜
5,⚫,⚫,❌,⚫,⚫,⚫,⬜,⬜,⬜,⚫
6,⚫,⚫,❌,⚫,⚫,⚫,⚫,⚫,⬜,⚫
7,❌,⚫,❌,⚫,❌,❌,❌,⚫,⚫,⚫
8,❌,⚫,⚫,⚫,⚫,⚫,⚫,⚫,⚫,⚫
9,⚫,⬜,⚫,⚫,⬜,⚫,⚫,⚫,⬜,⬜
10,⚫,⚫,⚫,⬜,⚫,⬜,⬜,⬜,⚫,⚫


,A,B,C,D,E,F,G,H,I,J
1,⬜,⚫,⚫,⚫,⚫,⚫,⬜,⬜,⚫,⚫
2,⚫,❌,⚫,⚫,⚫,⚫,⬜,⬜,⚫,⚫
3,⚫,❌,❌,⚫,❌,❌,❌,❌,⚫,⚫
4,⚫,❌,❌,⚫,⚫,⚫,⬜,⬜,⬜,⬜
5,⚫,⚫,❌,⚫,⚫,⚫,⬜,⬜,⬜,⚫
6,⚫,⚫,❌,⚫,⚫,⚫,⚫,⚫,⬜,⚫
7,❌,⚫,❌,⚫,❌,❌,❌,⚫,⚫,⚫
8,❌,⚫,⚫,⚫,⚫,⚫,⚫,⚫,⚫,⚫
9,⚫,⬜,⚫,⚫,⬜,⚫,⚫,⚫,⬜,⬜
10,⚫,⚫,⚫,⬜,⚫,⬜,⬜,⬜,⚫,⚫


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬛,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬛,⬛,⬜,⬛,⬛,⬛,⬛,⬜,⬜
4,⬜,⬛,⬛,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬛,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⬜,⬛,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,⬛,⬜,⬛,⬜,⬛,⬛,⬛,⬜,⬜,⬜
8,⬛,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
10,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


Completed in 79 steps
